In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [7]:
import numpy as np
import pandas as pd

data=pd.read_csv('//kaggle/input/digit-recognizer/train.csv')

In [8]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

In [14]:
data_dev = data[0:1000].T
y_dev = data_dev[0]
x_dev = data_dev[1:n]
x_dev = x_dev / 255.

data_train = data[1000:m].T
y_train = data_train[0]
x_train = data_train[1:n]
x_train = x_train / 255.
_,m_train = x_train.shape

In [10]:
Y_train

array([8, 6, 5, ..., 8, 4, 0])

In [23]:
def init_params():
    w1=np.random.rand(10,784)-0.5
    b1=np.random.rand(10,1)-0.5
    w2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    return w1,b1,w2,b2

def ReLU(z):
    return np.maximum(0,z)
    
def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def forward_prop(w1,b1,w2,b2,x):
    z1=w1.dot(x)+b1
    a1=ReLU(z1)
    z2=w2.dot(a1)+b2
    a2=softmax(z1)
    return z1,a1,z2,a2

def onehot(y):
    onehot_y=np.zeros((y.size,y.max()+1))
    onehot_y[np.arange(y.size),y]=1
    onehot_y=onehot_y.T
    return onehot_y

def deriv_ReLU(z):
    return z>0
    
def back_prop(z1,a1,z2,a2,w2,x,y):
    m=y.size
    onehot_y=onehot(y)
    dz2=a2-onehot_y
    dw2=1/m * dz2.dot(a1.T)
    db2=1/m * np.sum(dz2)
    
    dz1=w2.T.dot(dz2) * deriv_ReLU(z1)
    dw1=1/m * dz2.dot(x.T)
    db1=1/m * np.sum(dz2)
    
    return dw1,db1,dw2,db2

def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,a):
    w1=w1-a*dw1
    b1=b1-a*db1
    w2=w2-a*dw2
    b2=b2-a*db2
    return w1,b1,w2,b2

In [16]:
def get_predictions(a):
    return np.argmax(a,0)

def get_accuracy(pred,y):
    print(pred,y)
    return np.sum(pred==y)/y.size

def gradient_descent(x,y,iteration,a):
    w1,b1,w2,b2=init_params()
    for i in range(iteration):
        z1,a1,z2,a2=forward_prop(w1,b1,w2,b2,x)
        dw1,db1,dw2,db2=back_prop(z1,a1,z2,a2,w2,x,y)
        w1,b1,w2,b2=update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,a)
        if i%50==0:
            print("Iteration: ",i)
            print("Accuracy: ",get_accuracy(get_predictions(a2),y))
    return w1,b1,w2,b2

In [24]:
w1,b1,w2,b2=gradient_descent(x_train,y_train,500,0.1)

Iteration:  0
[7 6 6 ... 7 6 5] [1 2 2 ... 1 7 6]
Accuracy:  0.1345609756097561
Iteration:  50
[1 2 6 ... 1 7 5] [1 2 2 ... 1 7 6]
Accuracy:  0.615
Iteration:  100
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.7335121951219512
Iteration:  150
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.7794634146341464
Iteration:  200
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.804829268292683
Iteration:  250
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.8223170731707317
Iteration:  300
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.832390243902439
Iteration:  350
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.8409268292682927
Iteration:  400
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.848170731707317
Iteration:  450
[1 2 2 ... 1 7 6] [1 2 2 ... 1 7 6]
Accuracy:  0.8543170731707317


In [25]:
def make_pred(x,w1,b1,w2,b2):
    _,_,_,a2=forward_prop(w1,b1,w2,b2,x)
    return get_predictions(a2)

def test_pred(idx,w1,b1,w2,b2):
    curr_img=x_train[:,idx,None]
    pred=make_pred(x_train[:,idx,None],w1,b1,w2,b2)
    label=y_train[idx]
    print("Predictions: ",pred)
    print("Label: ",label)
    
    curr_img=curr_img.reshape((28,28))*255
    plt.gray()
    plt.imshow(curr_img,interpolation='nearest')
    plt.show()

In [26]:
dev_pred=make_pred(x_dev,w1,b1,w2,b2)
get_accuracy(dev_pred,y_dev)

[2 1 1 2 4 7 6 2 0 3 3 1 7 3 8 7 7 9 6 1 5 1 5 8 7 6 9 7 0 9 5 1 3 8 2 9 3
 9 3 0 5 7 6 1 6 2 0 5 3 1 0 2 5 8 7 4 9 3 1 4 3 3 6 8 5 7 3 5 3 7 8 9 3 4
 7 9 8 6 6 9 7 7 6 5 1 7 6 7 7 5 6 2 6 6 2 9 0 7 9 9 8 9 1 8 7 1 1 0 8 7 0
 6 9 1 4 5 9 0 7 1 4 2 2 8 1 1 0 9 2 2 2 8 9 9 8 3 4 9 9 6 1 0 6 7 4 7 6 2
 6 5 3 4 9 4 1 2 2 9 0 6 1 1 3 8 4 4 5 2 8 8 5 7 4 3 2 6 2 6 1 7 4 0 2 4 5
 7 9 1 8 7 1 0 1 4 3 8 8 6 1 5 1 1 0 0 5 8 1 6 9 5 9 0 5 8 4 1 8 8 9 1 0 1
 7 0 9 3 9 9 6 5 1 2 0 5 7 4 7 2 1 9 2 8 2 7 7 8 4 7 5 9 4 5 3 6 6 1 1 3 6
 5 7 7 1 1 9 6 3 2 9 4 4 0 6 6 7 2 8 5 6 9 9 4 8 3 1 2 8 5 3 7 0 1 9 0 9 3
 6 2 7 0 5 9 7 7 8 6 0 6 1 2 8 7 7 7 1 6 9 7 0 3 0 6 0 0 4 4 6 4 1 2 9 3 9
 0 0 7 0 2 2 4 9 5 3 5 0 8 4 5 5 1 4 1 8 2 9 8 1 0 9 0 4 3 9 7 4 9 4 8 1 0
 5 0 4 8 8 1 3 8 7 6 9 5 5 9 2 7 8 9 3 3 3 8 0 6 2 5 6 6 8 7 1 9 4 9 2 4 4
 6 3 8 2 6 9 7 7 8 5 7 7 0 1 5 9 6 3 1 5 1 6 9 4 5 2 8 3 8 6 3 8 0 8 5 6 6
 9 4 2 6 1 7 4 2 5 0 4 1 7 8 1 5 4 5 7 0 0 7 3 9 3 4 4 7 9 9 2 1 0 3 6 1 4
 3 6 4 0 0 5 8 7 9 3 6 5 

0.866